## 1. Packages

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
import json
import pandas as pd
import os
from openai import OpenAI
from IPython.display import display, HTML

## 2. ~~FUNCIÓN: Carga de todos los JSON en un DataFrame para su visualización~~

In [ ]:

# def crear_dataframe_json(json_path):
#     with open(json_path, 'r', encoding='utf-8') as file:
#         data = json.load(file)
    
#     nombre = data.get("name", "Desconocido")
#     texto = data.get("text", [])
    
#     # Crear diccionario base
#     df_data = {"name": [nombre]}
    
#     # Procesar páginas
#     for entry in texto:
#         page_num = entry.get("page", 0)
#         content = entry.get("content", "")
#         embedding = entry.get("OpenAI_embedding", [])
        
#         df_data[f"page_{page_num}"] = [page_num]
#         df_data[f"content_{page_num}"] = [content]
#         df_data[f"embedding_{page_num}"] = [embedding]
    
#     # Crear DataFrame
#     df = pd.DataFrame(df_data)
#     return df

In [ ]:
import os
import json
import pandas as pd

def crear_dataframe_json(folder_path):
    '''
    Esta función crea un DataFrame a partir de archivos JSON ubicados en un directorio específico.
    
    Cada JSON representa información extraída de un PDF, estructurada de la siguiente manera:
    
    1. Contiene un nombre de archivo que identifica el documento.
    2. Incluye una lista de páginas con:
        - Número de página (`page`)
        - Contenido de la página (`content`)
        - Representación numérica del contenido generada por OpenAI (`OpenAI_embedding`)

    La función recorrerá todos los archivos JSON dentro del `folder_path`, extraerá la información de cada uno,
    y generará un DataFrame donde cada fila representa un documento con sus respectivas páginas indexadas.
    '''

    all_data = []  # Lista para almacenar los datos procesados

    # Itera sobre los archivos en la carpeta proporcionada
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):  # Filtra solo archivos JSON
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)  # Carga el contenido del JSON
            
            # Obtiene el nombre del documento (por defecto, usa el nombre del archivo)
            nombre = data.get("pdf_filename", filename)
            texto = data.get("text", [])  # Lista de páginas con su contenido
            
            df_data = {"name": nombre}  # Diccionario base para almacenar los datos del documento
            # Procesa cada página del JSON
            for entry in texto:
                page_num = entry.get("page", 0)  # Obtiene el número de página
                df_data[f"page_{page_num}"] = page_num  # Almacena el número de página
                df_data[f"content_{page_num}"] = entry.get("content", "")  # Almacena el texto de la página
                df_data[f"embedding_{page_num}"] = entry.get("OpenAI_embedding", [])  # Almacena el embedding si está disponible
            
            all_data.append(df_data)  # Agrega la información procesada a la lista
    
    df = pd.DataFrame(all_data)  # Convierte la lista en un DataFrame
    return df  # Devuelve el DataFrame resultante


In [ ]:
# ejemplo de uso
path = "../assets/DG_docs/jsonembedding_pdf/"
df = crear_dataframe_json(path)

In [ ]:
df.head(3)
#df.style.set_sticky()

### 2.1. Del DataFrame anterior, selecciono las primeras 4 columnas que representan la "página1"

In [ ]:
df_subset = df[['name', 'page_1','content_1','embedding_1']]

In [ ]:
df_subset.head(3)

### 2.2. Del DataFrame anterio, se eliminan los `nan`

In [ ]:
# AVERIGUAR POR QUÉ HAY nan DENTRO DE LOS EMBEDDINGS
df_cleaned = df_subset.dropna(subset=["embedding_1"]).copy()

display(type(df_cleaned['embedding_1'][0]),len(df_cleaned['embedding_1'][0]))

df_cleaned['ids'] = df_cleaned.index.astype(str)
df_cleaned["embedding_1"] = df_cleaned["embedding_1"].astype(str)

#df_cleaned = df.dropna(subset=["embedding_1"])

In [ ]:
df_cleaned.head(3)

In [ ]:
# def save_df_to_csv(df, filename='output.csv', index=False):
#     """
#     Guarda un DataFrame como un archivo CSV.
    
#     :param df: DataFrame de Pandas a guardar.
#     :param filename: Nombre del archivo CSV de salida (por defecto 'output.csv').
#     :param index: Si se debe incluir el índice en el archivo CSV (por defecto False).
#     """
#     try:
#         df.to_csv(filename, index=index, encoding='utf-8')
#         print(f"DataFrame guardado correctamente en {filename}")
#     except Exception as e:
#         print(f"Error al guardar el DataFrame: {e}")


# save_df_to_csv(df, 'datos.csv')

In [ ]:
# sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = 'all-MiniLM-L6-v2')

In [ ]:
#df_cleaned['embedding_1'][0]

In [ ]:
#display(type(df_cleaned['embedding_1'][0]))

## 3. FUNCIÓN:  Genera un embedding del texto usando OpenAI
ESTÁ MAL COMENTADA LA FUNCIÓN

In [ ]:
def generar_embedding(texto: str) -> list:
    """
    Genera un embedding del texto usando OpenAI.

    WARNING:
        
    Packages:
        from openai import OpenAI
        import os
        import json

    Args:
        texto: Texto a procesar para obtener su embedding.

    Returns:
        dict: Un diccionario con el embedding generado y la cantidad de tokens procesados.

    Example:
        generar_embedding("Este es un texto de prueba")
    """
    
    try:
        # Llama a la API de OpenAI para obtener embeddings
        client = OpenAI()
        respuesta = client.embeddings.create(
            input=texto[:8191],  # OpenAI limita a 8192 tokens por entrada
            # EL COMENTARIO ANTERIOR ESTA MAL ENFORACADO PORQUE SE RECORTA EL TEXTO A 8191 Y LO QUE SE DEBE RECORTAR ES HASTA 8191 TOKENS
            model="text-embedding-3-small"  # Modelo optimizado para embeddings
        )
    
    except Exception as e:
        print(f"Error generando embedding: {e}")
        return None
    
    return respuesta.data[0].embedding

## ~~Uso de ChromDB de forma local~~

In [ ]:
# import os
# import ast
# import chromadb
# from chromadb.utils import embedding_functions

# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#     api_key = os.environ.get("OPENAI_API_KEY"),
#     model_name = "text-embedding-3-small"
#     )

# # Inicializar el cliente de ChromaDB con almacenamiento persistente
# chroma_client_persistent = chromadb.PersistentClient(path="./chroma_db")

# # Crear o recuperar la colección
# collection = chroma_client_persistent.get_or_create_collection(name="documentos", embedding_function=openai_ef)

# # Procesar los datos y agregarlos a la colección de ChromaDB
# ids = df_cleaned['ids'].astype(str).tolist()
# #documents = df_cleaned['content_1'].tolist() #esto se coloca cuando no se tiene los embedings, sedcoloca `documents`
# embeddings = [ast.literal_eval(e) for e in df_cleaned["embedding_1"].astype(str)]
# metadatas = df_cleaned[["name", "page_1"]].to_dict("records")  #tipo diccionario

# display(ids)


# # Agregar los datos a la colección en ChromaDB
# collection.add(ids=ids, embeddings=embeddings, metadatas=metadatas)

# # Confirmación
# f"Se han agregado {len(ids)} documentos a ChromaDB."


La idea de esta prueba es cambiar la seccion anterior
## ~~PRUEBA O.K.: Almacenamiento de la información en ChromaDB (modo servidor) PERO usa la data de un DataFrame no del JSON directamente~~

```powershell
PS C:\Users\devel\UNAD\INVIAS\INVIAS_NLP> curl http://localhost:8000
curl: (7) Failed to connect to localhost port 8000 after 2235 ms: Couldn't connect to server
```

```powershell
PS C:\Users\devel\UNAD\INVIAS\INVIAS_NLP chroma run --path ./assets/DG_docs/chroma_db
```

```powershell
PS C:\Users\devel\UNAD\INVIAS\INVIAS_NLP> netstat -ano | findstr :8000
  TCP    127.0.0.1:8000         0.0.0.0:0              LISTENING       16144
  TCP    [::1]:8000             [::]:0                 LISTENING       16144
```


In [ ]:
# !chroma run --path ./assets/DG_docs/chroma_db
# 4️⃣ (Opcional) Ejecutar en segundo plano
# Si quieres que el servidor siga corriendo después de cerrar la terminal, usa:

# nohup chromadb run --path /var/lib/chroma_db/ > chroma.log 2>&1 &

# Esto mantendrá el proceso activo y guardará los logs en chroma.log.

#chromadb run --path="./assets/DG_docs/chroma_db" --host=0.0.0.0 --port=8000

# !netstat -ano | findstr :8000 
# !taskkill /PID 568 /F
# !curl http://localhost:8000


In [ ]:
# import os
# import ast
# import chromadb
# from chromadb.utils import embedding_functions

# # Configurar la función de embeddings de OpenAI
# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#     api_key=os.environ.get("OPENAI_API_KEY"),
#     model_name="text-embedding-3-small"
# )

# # Conectar al servidor de ChromaDB (ya no se usa PersistentClient ni Settings)
# chroma_http_client = chromadb.HttpClient(host="localhost", port=8000)

# # Crear o recuperar la colección
# collection = chroma_http_client.get_or_create_collection(
#     name="pdf_document",
#     embedding_function=openai_ef
# )

# # Procesar los datos
# ids = df_cleaned['ids'].astype(str).tolist()
# embeddings = [ast.literal_eval(e) for e in df_cleaned["embedding_1"].astype(str)]
# metadatas = df_cleaned[["name", "page_1"]].to_dict("records")

# # Agregar los datos a la colección
# collection.upsert(ids=ids, embeddings=embeddings, metadatas=metadatas)

# # Confirmación
# print(f"Se han agregado {len(ids)} documentos a ChromaDB.")


In [ ]:
# chroma_http_client.list_collections()

## borra la coleccion

In [ ]:
# import chromadb

# # Conexión al servidor de ChromaDB
# chroma_http_client = chromadb.HttpClient(host="localhost", port=8000)

# collection_name = "pdf_document"

# # Obtener la lista de nombres de colecciones (directamente)
# collections = chroma_http_client.list_collections()  # Ya devuelve solo nombres

# if collection_name in collections:
#     chroma_http_client.delete_collection(collection_name)
#     print(f"✅ La colección '{collection_name}' ha sido eliminada.")
# else:
#     print(f"⚠️ La colección '{collection_name}' no existe.")

## ~~PRUEBA OK: Almacenamiento de los JSON directamente a ChromaDB de _todas las hojas de un PDF_; a diferencia de la anterior que almacena en ChromaDB a partir de DataFrame~~

```powershell
PS C:\Users\devel\UNAD\INVIAS\INVIAS_NLP> curl http://localhost:8000
curl: (7) Failed to connect to localhost port 8000 after 2235 ms: Couldn't connect to server
```

```powershell
PS C:\Users\devel\UNAD\INVIAS\INVIAS_NLP chroma run --path ./assets/DG_docs/chroma_db
```

```powershell
PS C:\Users\devel\UNAD\INVIAS\INVIAS_NLP> netstat -ano | findstr :8000
  TCP    127.0.0.1:8000         0.0.0.0:0              LISTENING       16144
  TCP    [::1]:8000             [::]:0                 LISTENING       16144
```


In [1]:
import os
import json
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from chromadb.utils import embedding_functions
import ast


# Cargar el archivo JSON
def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

# Guardar en ChromaDB
def store_in_chromadb(json_data, collection_name="pdf_document"):
    # Inicializar el cliente de ChromaDB
    chroma_http_client = chromadb.HttpClient(host="localhost", port=8000)
    
    # Configurar la función de embeddings de OpenAI
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.environ.get("OPENAI_API_KEY"),
        model_name="text-embedding-3-small"
    )

    # Crear o recuperar la colección
    collection = chroma_http_client.get_or_create_collection(
        name                = collection_name,
        embedding_function  = openai_ef
    )
    
    pdf_filename = json_data["pdf_filename"]
    pages = json_data["text"]
    
    for page_data in pages:
        page_number = page_data["page"]
        content = page_data["content"]
        
        embedding = page_data["OpenAI_embedding"]

        #embedding = ast.literal_eval(e) for e in page_data["OpenAI_embedding"]
        
        collection.add(
            ids=[f"{pdf_filename}_page_{page_number}"],
            metadatas=[{"pdf_filename": pdf_filename, "page": page_number}],
            embeddings=[embedding],
            documents=[content]
        )


        # # Procesar los datos
        # ids = df_cleaned['ids'].astype(str).tolist()
        # embeddings = [ast.literal_eval(e) for e in df_cleaned["embedding_1"].astype(str)]
        # metadatas = df_cleaned[["name", "page_1"]].to_dict("records")

        # # Agregar los datos a la colección
        # collection.upsert(ids=ids, embeddings=embeddings, metadatas=metadatas)

    
    print(f"Documento '{pdf_filename}' almacenado con {len(pages)} páginas en ChromaDB.")

    # display("display(collection.peek(8), collection.count())",display(collection.peek(8), collection.count()) )

   

    # # Extraer y mostrar los IDs o metadatas o embeddings o documents
    
    # print(collection.get().keys())
    # display(collection.count())

    # display(collection.get()['ids'])
    # display(collection.get()['metadatas'])
    

    #display(collection.peek(1))
    #display(collection.peek(1)['embeddings'].shape)



In [4]:
# ejemplo de uso
json_path = "../assets/DG_docs/jsonembedding_pdf/2024S-VBOG-004040 (1).json"

df = store_in_chromadb(load_json(json_path))




Documento '2024S-VBOG-004040 (1).pdf' almacenado con 11 páginas en ChromaDB.


## 4. Queries a ChromaDB (modo servidor)

In [ ]:
def query_chromadb_remote(query_text, collection_name="pdf_document", top_k=3, host="localhost", port=8000):
    """
    Realiza una consulta en ChromaDB alojado en un servidor remoto.
    
    :param query_text: Texto de la consulta.
    :param collection_name: Nombre de la colección en ChromaDB.
    :param top_k: Número de resultados más relevantes a recuperar.
    :param host: Dirección del servidor remoto de ChromaDB.
    :param port: Puerto del servidor ChromaDB (por defecto 8000).
    :return: Resultados de la consulta.
    """
    
    # Conectar al servidor de ChromaDB (ya no se usa PersistentClient ni Settings)
    chroma_http_client = chromadb.HttpClient(host=host, port=port)
    
    # Configurar la función de embeddings de OpenAI
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.environ.get("OPENAI_API_KEY"),
        model_name="text-embedding-3-small"
    )
    
    
    collection = chroma_http_client.get_collection(name = collection_name, embedding_function = openai_ef)
    
    results = collection.query(
        query_texts=[query_text],
        n_results=top_k
        # where={"metadata_field": "is_equal_to_this"},
        # where_document={"$contains":"search_string"} # "$not_contains": "search_string"
    )
    
    return results


    # n_results=10,
    # where={"metadata_field": "is_equal_to_this"},
    # where_document={"$contains":"search_string"} # "$not_contains": "search_string"



In [ ]:
# ¿Cuál es el presupuesto del programa Caminos Comunitarios de la Paz Total?

query_chromadb_remote("¿Cuál es el presupuesto del programa Caminos Comunitarios de la Paz Total?")

## OTROS

In [ ]:
from chromadb.config import Settings
print(Settings.__annotations__)